In [1]:
!pip install pytz -q

In [2]:
import os
import json
from openai import AzureOpenAI
from datetime import datetime
from zoneinfo import ZoneInfo
import pytz
from dotenv import load_dotenv

In [5]:
endpoint = os.environ.get('ENDPOINT_URL')
deployment = os.environ.get('DEPLOYMENT_NAME')
api_key = os.environ.get('AZURE_OPENAI_KEY')

In [4]:
load_dotenv()

True

In [6]:
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2023-07-01-preview"
)

In [7]:
TIMEZONE_DATA = {
    "india": "Asia/kolkata",
    "tokyo": "Asia/Tokyo",
    "san francisco": "America/Los_Angeles",
    "paris": "Europe/Paris",
    
}

In [8]:
def get_current_time(location):
    """Get the current time for a given location"""
    print(f"get_current_time called with location: {location}") 
    location_lower = location.lower()
    
    for key, timezone in TIMEZONE_DATA.items():
        print(f"Checking key: {key} in location: {location_lower}")
        if key in location_lower:
            print(f"Timezone found for {key}")  
            tz = pytz.timezone(timezone)
            current_time = datetime.now(tz).strftime("%I:%M %p")
            return json.dumps({
                "location": location,
                "current_time": current_time
            })
    
    print(f"No timezone data found for {location_lower}")  
    return json.dumps({"location": location, "current_time": "unknown"})

In [10]:
print(get_current_time("india"))

get_current_time called with location: india
Checking key: india in location: india
Timezone found for india
{"location": "india", "current_time": "12:05 PM"}


In [9]:
# Define the function for the model
tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_time",
                "description": "Get the current time in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city name, e.g. San Francisco",
                        },
                    },
                    "required": ["location"],
                },
            }
        }
    ]

In [10]:
def call_openai_function_calling(messages, tools):
    response = client.chat.completions.create(
        model=deployment,
        messages=messages,
        tools = tools,
        tool_choice="auto",
        
    )
    return response

In [26]:
def run_conversation(prompt):
    # Initial user message
    messages = [{"role": "system", "content": "You are helpful asssitant were you provide Current time as per Given Location"},
               {"role" : "user" , "content": prompt}
               ] # Single function call
    

    # Process the model's response
    response = call_openai_function_calling(messages,tools)

    response_message = response.choices[0].message
    print(response_message)
    messages.append(response_message)
    
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            if tool_call.function.name == "get_current_time":
                function_args = json.loads(tool_call.function.arguments)
                print(f"Function arguments: {function_args}")  
                time_response = get_current_time(
                    location=function_args.get("location")
                )
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": "get_current_time",
                    "content": time_response,
                })
    else:
        print("No tool calls were made by the model.")  
        return response.choices[0].message.content

    # Second API call: Get the final response from the model
    final_response = call_openai_function_calling(messages,tools)

    return final_response.choices[0].message.content
    



In [28]:
while True :
    user_input = input("User: ")
    if user_input.lower() in ["exit","quit"]:
        print("Chat Session Ended!....")
        break
    response = run_conversation(user_input)
    print(f"Assistant: {response}")

User:  give me time for india


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_NEmHK7D51wE4iPuWkfsIFAOG', function=Function(arguments='{"location":"India"}', name='get_current_time'), type='function')])
Function arguments: {'location': 'India'}
get_current_time called with location: India
Checking key: india in location: india
Timezone found for india
Assistant: The current time in India is 12:24 PM.


User:  for San Francisco


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_GXwClVornYZBCaVN3BuO98dF', function=Function(arguments='{"location":"San Francisco"}', name='get_current_time'), type='function')])
Function arguments: {'location': 'San Francisco'}
get_current_time called with location: San Francisco
Checking key: india in location: san francisco
Checking key: tokyo in location: san francisco
Checking key: san francisco in location: san francisco
Timezone found for san francisco
Assistant: The current time in San Francisco is 11:54 PM.


User:  quit


Chat Session Ended!....


In [19]:
print(type(run_conversation))

<class 'function'>
